In [9]:
import pandas as pd
from hdbcli import dbapi

# Configuración de conexión
host = "10.10.6.132"  # IP o nombre del servidor SAP HANA
port = 30015  # Puerto estándar de HANA
user = "B1ADMIN"
password = "Pt3.4dm1n"

# Crear conexión
conn = dbapi.connect(
    address=host,
    port=port,
    user=user,
    password=password
)

In [10]:
# Consulta SQL

df_clientes = pd.read_excel("C:\\Users\\Juan Avendaño\\Downloads\\clientes fabiola febrero.xlsx")

resultados = []

for cliente in df_clientes["Clientes"]:
    query = f"""
    SELECT DISTINCT
        T0."validFor",
        T0."CardCode" AS "NIT",
        T0."CardName" AS "Cliente",
        T0."CreateDate" AS "Fecha de creación",
        (
            SELECT
                MAX(T2."DocDate")
            FROM
                BD_PARTEQUIPOS_PRO.OINV T2
            WHERE
                T2."CardCode" = T0."CardCode"
        ) AS "Ultima factura",
        (
            SELECT
                MAX(T2."DocDate")
            FROM
                BD_PARTEQUIPOS_PRO.OQUT T2
            WHERE
                T2."CardCode" = T0."CardCode"
        ) AS "Ultima Cotización",
        T0."Address" AS "Dirección",
        T0."City" AS "Ciudad",
        T3."Name" AS "Municipio",
        T4."FirstName" || ' ' || T4."MiddleName" || ' ' || T4."LastName" AS "Persona contacto",
        T0."Phone1" AS "Teléfono 1",
        T0."Phone2" AS "Teléfono 2",
        T0."Cellular" AS "Celular",
        T0."E_Mail" AS "Correo electrónico",
        T2."SlpName" AS "Asesor",
        T0."U_vendedor1" AS "Vendedor 1",
        (
        SELECT SUM(T6."LineTotal") -- Ajusta "LineTotal" si el campo correcto es otro
        FROM BD_PARTEQUIPOS_PRO.OINV T5
        INNER JOIN BD_PARTEQUIPOS_PRO.INV1 T6 ON T5."DocEntry" = T6."DocEntry"
        INNER JOIN BD_PARTEQUIPOS_PRO.OITM T7 ON T6."ItemCode" = T7."ItemCode"
        WHERE T5."CardCode" = T0."CardCode"
        AND T7."ItmsGrpCod" IN (101,113)
        ) AS "Total Ventas Filtros",
        (
        SELECT SUM(T6."LineTotal") -- Ajusta "LineTotal" si el campo correcto es otro
        FROM BD_PARTEQUIPOS_PRO.OINV T5
        INNER JOIN BD_PARTEQUIPOS_PRO.INV1 T6 ON T5."DocEntry" = T6."DocEntry"
        INNER JOIN BD_PARTEQUIPOS_PRO.OITM T7 ON T6."ItemCode" = T7."ItemCode"
        WHERE T5."CardCode" = T0."CardCode"
        AND T7."ItmsGrpCod" IN (102,114)
        ) AS "Total Ventas Lubricantes"
    FROM
        "BD_PARTEQUIPOS_PRO".OCRD T0
        INNER JOIN "BD_PARTEQUIPOS_PRO".OSLP T2 ON T0."SlpCode" = T2."SlpCode"
        LEFT JOIN "BD_PARTEQUIPOS_PRO"."@BPCO_MU" T3 ON T3."Code" = T0."U_BPCO_CS"
        LEFT JOIN "BD_PARTEQUIPOS_PRO".OCPR T4 ON T0."CardCode" = T4."CardCode"
    WHERE
        T0."CardType" = 'C'
        AND T0."CardName" LIKE '%{cliente}%'
    """

# Cargar datos en Pandas
    df = pd.read_sql(query, conn)
    resultados.append(df)
    
df_final = pd.concat(resultados, ignore_index=True)

# Guardar en un nuevo archivo de Excel
df_final.to_excel("Clientes_Resultado.xlsx", index=False)

C:\Users\Juan Avendaño\AppData\Local\Temp\ipykernel_29468\1412429238.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Juan Avendaño\AppData\Local\Temp\ipykernel_29468\1412429238.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Juan Avendaño\AppData\Local\Temp\ipykernel_29468\1412429238.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Juan Avendaño\AppData\Local\Temp\ipykernel_29468\1412429238.py:6

In [11]:
# Cerrar conexión
conn.close()

# Mostrar resultados
print(df.head())

Empty DataFrame
Columns: [validFor, NIT, Cliente, Fecha de creación, Ultima factura, Ultima Cotización, Dirección, Ciudad, Municipio, Persona contacto, Teléfono 1, Teléfono 2, Celular, Correo electrónico, Asesor, Vendedor 1, Total Ventas Filtros, Total Ventas Lubricantes]
Index: []
